# Get Online Inference from Vertex AI Endpoint

This notebook demonstrates how to get predictions from a deployed Vertex AI Endpoint.
Make sure you have authenticated with Google Cloud before running this.


In [1]:
# Import Required Libraries
from google.cloud import aiplatform
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

print("Libraries imported successfully.")

Libraries imported successfully.


## Train Model and Create Joblib Files

Before getting online inference, we need to train the model and save it as joblib files.


In [2]:
# Load the women's clothing e-commerce reviews dataset
df = pd.read_csv("../womens_clothing_ecommerce_reviews.csv")

print("Dataset loaded successfully.")
print(f"Dataset shape: {df.shape}")
print(df.head())

Dataset loaded successfully.
Dataset shape: (19818, 2)
                                         Review Text  sentiment
0  Absolutely wonderful - silky and sexy and comf...          1
1  Love this dress!  it's sooo pretty.  i happene...          1
2  I love, love, love this jumpsuit. it's fun, fl...          1
3  This shirt is very flattering to all due to th...          1
4  I love tracy reese dresses, but this one is no...         -1


In [3]:
# Define features (X) and target variable (Y)
X = df['Review Text']
Y = df['sentiment']

# Verify data types
print(type(X))
print(type(Y))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [4]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

print("\nShapes of Train/Test Datasets:")
print("X_train shape:", x_train.shape)
print("Y_train shape:", y_train.shape)
print("X_test shape:", x_test.shape)
print("Y_test shape:", y_test.shape)


Shapes of Train/Test Datasets:
X_train shape: (15854,)
Y_train shape: (15854,)
X_test shape: (3964,)
Y_test shape: (3964,)


In [5]:
# Initialize and train CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
x_train_bow = vectorizer.fit_transform(x_train)
x_test_bow = vectorizer.transform(x_test)

print("\nText vectorization completed.")
print(x_train_bow)
print("Training set shape:", x_train_bow.shape)
print("Testing set shape:", x_test_bow.shape)


Text vectorization completed.
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 373991 stored elements and shape (15854, 11897)>
  Coords	Values
  (0, 10214)	1
  (0, 2396)	1
  (0, 4805)	1
  (0, 4210)	1
  (0, 3835)	1
  (0, 9289)	1
  (0, 3680)	1
  (0, 4669)	1
  (0, 6980)	1
  (0, 10773)	1
  (0, 7248)	1
  (0, 7409)	1
  (1, 9289)	1
  (1, 2866)	1
  (1, 3454)	1
  (1, 4206)	1
  (1, 1781)	1
  (1, 6221)	1
  (1, 6711)	1
  (1, 8861)	1
  (1, 9671)	1
  (1, 9424)	1
  (1, 11)	1
  (2, 7409)	1
  (2, 3454)	3
  :	:
  (15853, 4805)	1
  (15853, 7596)	1
  (15853, 6257)	1
  (15853, 2390)	1
  (15853, 11495)	1
  (15853, 1602)	1
  (15853, 8053)	1
  (15853, 4742)	1
  (15853, 8467)	1
  (15853, 7829)	1
  (15853, 9466)	1
  (15853, 10594)	1
  (15853, 11408)	1
  (15853, 11414)	1
  (15853, 8195)	1
  (15853, 584)	1
  (15853, 11438)	1
  (15853, 5070)	1
  (15853, 11519)	1
  (15853, 9280)	1
  (15853, 9273)	1
  (15853, 5103)	1
  (15853, 3990)	1
  (15853, 521)	1
  (15853, 4857)	1
Training set shape: (15854, 11897)

In [6]:
# Train the Logistic Regression model
model = LogisticRegression(max_iter=5000, random_state=42, solver='saga')
model.fit(x_train_bow, y_train)

print("\nModel Training Completed.")


Model Training Completed.


In [7]:
# Make predictions and evaluate the model
y_pred = model.predict(x_test_bow)
accuracy = accuracy_score(y_test, y_pred)

print("\nModel Accuracy on Test Set: {:.2f}%".format(accuracy * 100))


Model Accuracy on Test Set: 92.99%


In [8]:
# Save the model and vectorizer as joblib files
joblib.dump(model, 'model.joblib')
joblib.dump(vectorizer, 'vectorizer.joblib')

print("\n✅ Model saved as 'model.joblib'")
print("✅ Vectorizer saved as 'vectorizer.joblib'")
print("\nThese files can now be used for online inference.")


✅ Model saved as 'model.joblib'
✅ Vectorizer saved as 'vectorizer.joblib'

These files can now be used for online inference.


## 1. Configuration

Please fill in your Project ID, Region, and the Display Name of your Endpoint.


In [9]:
# --- TODO: Update these values ---
PROJECT_ID = "YOUR_PROJECT_ID"  # e.g. "my-awesome-project"
REGION = "us-central1"          # e.g. "us-central1"
ENDPOINT_DISPLAY_NAME = "iris-endpoint-ui" # The name you gave your endpoint

print(f"Project: {PROJECT_ID}")
print(f"Region: {REGION}")
print(f"Endpoint Name: {ENDPOINT_DISPLAY_NAME}")

Project: YOUR_PROJECT_ID
Region: us-central1
Endpoint Name: iris-endpoint-ui


In [10]:
## 2. Initialize Vertex AI SDK
aiplatform.init(project=PROJECT_ID, location=REGION)

print("Vertex AI SDK Initialized.")

Vertex AI SDK Initialized.


In [11]:
## 3. Get Endpoint Object
# Look up the endpoint by its display name
endpoints = aiplatform.Endpoint.list(filter=f'display_name="{ENDPOINT_DISPLAY_NAME}"')

if endpoints:
    endpoint = endpoints[0]
    print(f"✅ Found Endpoint: {endpoint.resource_name}")
else:
    print(f"❌ Endpoint '{ENDPOINT_DISPLAY_NAME}' not found in region {REGION}.")
    print("Please check the name and region.")

c:\Users\chamm\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\auth\_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


PermissionDenied: 403 Permission denied on resource project YOUR_PROJECT_ID. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "YOUR_PROJECT_ID"
}
metadata {
  key: "consumer"
  value: "projects/YOUR_PROJECT_ID"
}
, locale: "en-US"
message: "Permission denied on resource project YOUR_PROJECT_ID."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
]

## 4. Prepare Test Data

We will define both a **Positive** review and a **Negative** review to test the model.


In [ ]:
# Example reviews (Positive and Negative)
test_reviews = [
    "The product was fantastic and arrived early!", # Positive
    "The product was terrible, broken, and arrived late." # Negative
]

# PRE-PROCESSING (Only for Text Models)
try:
    vectorizer = joblib.load("vectorizer.joblib")
    print("Step 1: Transforming text...")
    sparse_matrix = vectorizer.transform(test_reviews)

    print("Step 2: Converting to list...")
    # Vertex AI expects python lists, not numpy arrays
    prediction_instances = sparse_matrix.toarray().tolist()

    print(f"Data prepared. {len(prediction_instances)} instances ready.")

except FileNotFoundError:
    print("⚠️ 'vectorizer.joblib' not found. Skipping text processing.")
    print("Using dummy numeric data (for Iris model example):")

    # Iris Example Data (Class 0 and Class 2)
    prediction_instances = [
        [5.1, 3.5, 1.4, 0.2],
        [6.7, 3.0, 5.2, 2.3]
    ]
    print(f"Input: {prediction_instances}")

In [ ]:
## 5. Get Prediction

# Make the prediction request
print("Sending prediction request...")
try:
    response = endpoint.predict(instances=prediction_instances)

    print("\n--- Prediction Results ---")
    for i, prediction in enumerate(response.predictions):
        print(f"Review {i+1}: {test_reviews[i] if 'test_reviews' in locals() else 'Instance ' + str(i+1)}")
        print(f"Prediction: {prediction} \n")

except NameError:
    print("Endpoint not found. Cannot predict.")
except Exception as e:
    print(f"Error during prediction: {str(e)}")